In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (134 kB/s)
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
home_sales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

home_sales.createOrReplaceTempView("home_sales")
home_sales.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql import SQLContext

sqlContext = SQLContext(spark)

home_sales.createOrReplaceTempView("home_sales")

average_price_per_year_four_bed = spark.sql("""
    SELECT year(date) AS year, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year(date)
    ORDER BY year
""")

# Show the result
average_price_per_year_four_bed.show()

/content/spark-3.4.2-bin-hadoop3/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
sqlContext = SQLContext(spark)

average_price_per_year_3_bed_bath = spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY YEAR(date)
    ORDER BY year
""")

average_price_per_year_3_bed_bath.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|    287287.82|
|2020|    294204.16|
|2021|    294211.46|
|2022|    292725.69|
+----+-------------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

sqlContext = SQLContext(spark)

average_price_per_year_3_bed_bath_2_floor_sqf = spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY YEAR(date)
    ORDER BY year
""")

average_price_per_year_3_bed_bath_2_floor_sqf.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|    289859.14|
|2020|    292289.09|
|2021|    296330.96|
|2022|    290242.99|
+----+-------------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

# Start time measurement
start_time = time.time()

# Create SQL context
sqlContext = SQLContext(spark)

# Execute SQL query
average_price_per_view = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
""")

average_price_per_view.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.9276163578033447 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

# Execute SQL query
average_price_per_view = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
""")

average_price_per_view.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.5779633522033691 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales.write.partitionBy("date_built").parquet("home_sales_p.parquet")

In [ ]:
# 11. Read the parquet formatted data.
home_sales_parq = spark.read.parquet("home_sales_p.parquet")

In [ ]:
# 12. Create a temporary table for the parquet data.
home_sales_parq.createOrReplaceTempView("home_sales_parq")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

# Execute SQL query on Parquet DataFrame
average_price_per_view_parquet = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales_parq
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
""")

# Show the result
average_price_per_view_parquet.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 2.2118747234344482 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
home_sales.unpersist()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [ ]:
# 15. Check if the home_sales is no longer cached
if not home_sales.is_cached:
    print("home_sales DataFrame is no longer cached.")
else:
    print("home_sales DataFrame is still cached.")

home_sales DataFrame is no longer cached.
